## II/ Assesments Data

MDS-UPDRS_Part_III_03Oct2025.csv contains a  
DtypeWarning: Columns (16,21) have mixed types. Specify dtype option on import or set low_memory=False

#### Imports

In [1]:
import pandas as pd
import numpy as np

import os

from sklearn.impute import SimpleImputer
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

##### Number of unique patients per csv_file

In [2]:
all_csv_folder = "Dataset/CSV/all_csvs"

# Find all CSV files in that folder
csv_files = [f for f in os.listdir(all_csv_folder) if f.endswith(".csv")]

for f in csv_files:
    path = os.path.join(all_csv_folder, f)
    # try:
    df = pd.read_csv(path, dtype={16: "string", 21: "string"})

    patno_dtype = df["PATNO"].dtype if "PATNO" in df.columns else "❌ Not found"
    event_dtype = df["EVENT_ID"].dtype if "EVENT_ID" in df.columns else "❌ Not found"

    # print(f"{f}: PATNO → {patno_dtype}, EVENT_ID → {event_dtype}")
    # except Exception as e:
        # print(f"{f}: ❌ Error loading file → {e}")

for file, df in zip(csv_files, dfs):
    print(f'{file} contains: {len(df["PATNO"].unique())} patients.')

NameError: name 'dfs' is not defined

#### process the UPSIT csv

In [213]:
upsit_csv_path = "Dataset/CSV/Assesments/University_of_Pennsylvania_Smell_Identification_Test_UPSIT_03Oct2025.csv"
upsit_df = pd.read_csv(upsit_csv_path, dtype={"PATNO": str, "EVENT_ID": str})


# Columns to drop
cols_to_drop = ["upsitorder", "UPSITFORM", "UPSIT_PRCNTGE", "UPSIT_PRCTVER", "IMPUTED_DATA", "UPSIT_SOURCE", "UPSITLANGCNTR"]

# Drop the 40 SCENT_*_RESPONSE and SCENT_*_CORRECT columns
scent_cols_to_drop = [col for col in upsit_df.columns if col.startswith("SCENT_") and (
    col.endswith("_RESPONSE") or col.endswith("_CORRECT")
)]
# ---or just the _REPSONSE columns---
# response_cols = [c for c in upsit.columns if c.endswith("_RESPONSE")]

all_cols_to_drop = cols_to_drop + scent_cols_to_drop
upsit_df.drop(columns=[col for col in all_cols_to_drop if col in upsit_df.columns], inplace=True)


# Save back to same CSV
upsit_df.to_csv(upsit_csv_path, index=False)
print(f"\nCSV updated and saved successfully! Dropped {len(all_cols_to_drop)} columns.")
print("UPSIT cleaned shape:", upsit_df.shape)


CSV updated and saved successfully! Dropped 87 columns.
UPSIT cleaned shape: (7930, 8)


#### load and merge data

In [153]:
csv_folder = "Dataset/CSV/Assesments_old"

# Find all CSV files in that folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

# Columns to drop (repeated meta columns)
useless_cols = ['REC_ID', 'PAG_NAME', 'INFODT', 'ORIG_ENTRY', 'LAST_UPDATE']


dfs = []
for file in csv_files:
    # df = pd.read_csv(os.path.join(csv_folder, file), dtype=str, low_memory=False, dtype={"PATNO": str, "EVENT_ID": str})
    df = pd.read_csv(os.path.join(csv_folder, file), dtype={16: "string", 21: "string"})

    df.drop(columns=[col for col in useless_cols if col in df.columns], inplace=True)

    dfs.append(df)


# Merge all DataFrames on 'patient ID' and 'event ID'
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['PATNO', 'EVENT_ID'], how='inner')


# Drop columns ending with _x or _y (duplicates)
cols_to_drop = [
    col for col in merged_df.columns
    if (col.endswith('_x') or col.endswith('_y'))
]

merged_df.drop(columns=cols_to_drop, inplace=True)


print(f"Merged Shape: {merged_df.shape}\n")
print(f"Merged Columns: {merged_df.columns}\n")


Merged Shape: (10213, 162)

Merged Columns: Index(['PATNO', 'EVENT_ID', 'ESS1', 'ESS2', 'ESS3', 'ESS4', 'ESS5', 'ESS6',
       'ESS7', 'ESS8',
       ...
       'SCAU24', 'SCAU25', 'SCAU26A', 'SCAU26AT', 'SCAU26B', 'SCAU26BT',
       'SCAU26C', 'SCAU26CT', 'SCAU26D', 'SCAU26DT'],
      dtype='object', length=162)



#### head

In [47]:
merged_df.head()

,PATNO,EVENT_ID,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,...,SCAU24,SCAU25,SCAU26A,SCAU26AT,SCAU26B,SCAU26BT,SCAU26C,SCAU26CT,SCAU26D,SCAU26DT
0,3001,V10,1,2.0,1.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,1.0,vesicare,1.0,lisinopril,1.0,"finacea,metronidazole,clelopirox dlaminde"
1,3001,V10,1,2.0,1.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,1.0,vesicare,1.0,lisinopril,1.0,"finacea,metronidazole,clelopirox dlaminde"
2,3001,V12,1,2.0,1.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,1.0,"miralax,colace",1.0,vesicare,1.0,lisinopril,1.0,"lipitor,isradipine,lamisil,"
3,3001,V12,1,2.0,1.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,1.0,"miralax,colace",1.0,vesicare,1.0,lisinopril,1.0,"lipitor,isradipine,lamisil,"
4,3001,V14,1,2.0,0.0,0.0,2.0,0.0,1.0,0.0,...,NaN,NaN,1.0,colace,1.0,trospium chloride er,1.0,lisinopril,1.0,"isradipine (Raynaud's), atorvastatin (hypertip..."


#### info

In [48]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10213 entries, 0 to 10212
Columns: 162 entries, PATNO to SCAU26DT
dtypes: float64(135), int64(8), object(11), string(8)
memory usage: 12.6+ MB


#### describe

In [49]:
merged_df.describe(include="all")

,PATNO,EVENT_ID,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,...,SCAU24,SCAU25,SCAU26A,SCAU26AT,SCAU26B,SCAU26BT,SCAU26C,SCAU26CT,SCAU26D,SCAU26DT
count,10213.000000,10213,10213.000000,10210.000000,10211.000000,10213.000000,10213.000000,10211.000000,10209.000000,10200.000000,...,5681.000000,5674.000000,10208.000000,2987,10208.000000,1886,10207.000000,3510,10194.000000,3708
unique,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,785,NaN,567,NaN,1107,NaN,1850
top,NaN,V04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Miralax,NaN,Tamsulosin,NaN,Losartan,NaN,synthroid
freq,NaN,1515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,328,NaN,149,NaN,148,NaN,24
mean,47139.906981,NaN,1.248507,1.370911,0.610616,0.941056,1.936356,0.202037,0.851406,0.209216,...,5.688611,5.672718,0.296336,NaN,0.185541,NaN,0.346037,NaN,0.363940,NaN
std,64504.949192,NaN,0.934985,0.920716,0.803994,0.959446,0.995123,0.495642,0.892628,0.525973,...,4.059768,4.060473,0.456663,NaN,0.388755,NaN,0.475728,NaN,0.481155,NaN
min,3001.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
25%,3530.000000,NaN,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
50%,40533.000000,NaN,1.000000,1.000000,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000,...,9.000000,9.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
75%,56435.000000,NaN,2.000000,2.000000,1.000000,1.000000,3.000000,0.000000,1.000000,0.000000,...,9.000000,9.000000,1.000000,NaN,0.000000,NaN,1.000000,NaN,1.000000,NaN


#### nunique

In [50]:
merged_df.nunique().sort_values()

ONOFFORDER       2
OFFEXAM          2
DBSOFFYN         2
ONEXAM           2
DBSYN            2
              ... 
SCAU26AT       785
HRPOSTMED     1051
SCAU26CT      1107
PATNO         1335
SCAU26DT      1850
Length: 162, dtype: int64

### Order By Patiend ID

In [154]:
is_sorted = merged_df['PATNO'].is_monotonic_increasing
print(f"Is PATNO sorted? {is_sorted}")

Is PATNO sorted? True


In [155]:
merged_df.sort_values(by='PATNO', inplace=True)

### HOEHN and YAHR Stage

In [156]:
print(f"Merged Shape: {merged_df.shape}\n")

print(merged_df['NHY'].value_counts(dropna=False).sort_index())

Merged Shape: (10213, 162)

NHY
0.0       162
1.0      1380
2.0      6760
3.0       734
4.0       152
5.0        62
101.0      55
NaN       908
Name: count, dtype: int64


In [157]:
# Drop rows where NHY is NaN or '101'
merged_df = merged_df[~(merged_df['NHY'].isna() | (merged_df['NHY'] == '101'))]

# Merge '5' and '4' into one stage
merged_df['NHY'] = merged_df['NHY'].replace('5', '4')

In [158]:
print(f"Merged Shape: {merged_df.shape}\n")

print(merged_df['NHY'].value_counts(dropna=False).sort_index())

Merged Shape: (9305, 162)

NHY
0.0       162
1.0      1380
2.0      6760
3.0       734
4.0       152
5.0        62
101.0      55
Name: count, dtype: int64


### Patient's Functional State

In [159]:
print(f"Merged Shape: {merged_df.shape}\n")

print(merged_df['PDSTATE'].value_counts(dropna=False))

Merged Shape: (9305, 162)

PDSTATE
ON     5191
OFF    3693
NaN     421
Name: count, dtype: int64


In [160]:
merged_df = merged_df[~(merged_df['PDSTATE'] == 'ON')]

In [161]:
print(f"Merged Shape: {merged_df.shape}\n")

print(merged_df['PDSTATE'].value_counts(dropna=False))

Merged Shape: (4114, 162)

PDSTATE
OFF    3693
NaN     421
Name: count, dtype: int64


### 1. Handle Duplicates

In [162]:
print(f"BEFORE: Number of duplicate rows: {merged_df.duplicated().sum()}")

merged_df.drop_duplicates(inplace=True)
print(f"AFTER: Number of duplicate rows: {merged_df.duplicated().sum()}")

BEFORE: Number of duplicate rows: 0
AFTER: Number of duplicate rows: 0


### 2. Drop Useless/Sparse Columns

In [163]:
missing_values = merged_df.isnull().sum()
missing_percent = (missing_values / len(merged_df)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percent
})

missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage (%)', ascending=False)
missing_df

,Missing Values,Percentage (%)
HRDBSON,4114,100.000000
ONEXAM,4114,100.000000
ONNORSN,4114,100.000000
DBSOFFYN,4110,99.902771
OFFNORSN,4110,99.902771
...,...,...
SCAU3,1,0.024307
SCAU1,1,0.024307
SCAU4,1,0.024307
SCAU26B,1,0.024307


In [164]:
useless_cols = [
    'DBSOFFTM',
    'DBSONTM',
    'PTCGBOTH',
    'PDMEDDT',       # Date of medication start
    'PDMEDTM',       # Time of medication start
    'EXAMTM',        # Time of motor exam
]
print(f"Number of usless columns: {len(useless_cols)}")

sparse_cols = missing_df[missing_df['Percentage (%)'] > 55].index.tolist()
print(f"Number of sparse columns: {len(sparse_cols)}")

merged_df.drop(columns=sparse_cols + useless_cols, inplace=True)
print(f"New Shape: {merged_df.shape}\n")

Number of usless columns: 6
Number of sparse columns: 27
New Shape: (4114, 131)



### 3. Impute missing values

In [ ]:
# Separate numerical and categorical
numerical_cols = merged_df.select_dtypes(include='number').columns
categorical_cols = merged_df.select_dtypes(include='object').columns
print(f"Number of numerical columns: {len(numerical_cols)}")
print(f"Number of categorical columns: {len(categorical_cols)}\n")

# Print missing values before imputation
categorical_missing_before = merged_df[categorical_cols].isna().sum().sum()
print(f"Missing values in categorical columns before imputation: {categorical_missing_before}")

#? whole numerical imputation added when writing memoire
numerical_missing_before = merged_df[numerical_cols].isna().sum().sum()
print(f"Missing values in numerical columns before imputation: {numerical_missing_before}")

# Imputers
categorical_imputer = SimpleImputer(strategy='most_frequent')
numerical_imputer = SimpleImputer(strategy='mean') #? whole numerical imputation added when writing memoire 

# Apply
merged_df[categorical_cols] = categorical_imputer.fit_transform(merged_df[categorical_cols])
#? whole numerical imputation added when writing memoire 
merged_df[numerical_cols] = numerical_imputer.fit_transform(merged_df[numerical_cols])

# Print missing values after imputation
categorical_missing_after = merged_df[categorical_cols].isna().sum().sum()
print(f"Missing values in categorical columns after imputation: {categorical_missing_after}")

#? whole numerical imputation added when writing memoire 
numerical_missing_after = merged_df[numerical_cols].isna().sum().sum()
print(f"Missing values in numerical columns after imputation: {numerical_missing_after}")

Number of numerical columns: 122
Number of categorical columns: 3

Missing values in categorical columns before imputation: 423
Missing values in numerical columns before imputation: 8561
Missing values in categorical columns after imputation: 0
Missing values in numerical columns after imputation: 8561


### 4. Handle date columns

In [166]:
parsed_date_columns = []

for col in merged_df.columns:
    if merged_df[col].dtype == 'object':
        try:
            merged_df[col] = pd.to_datetime(merged_df[col], format='%Y-%m', errors='raise')
            parsed_date_columns.append(col)
        except:
            try:
                # Fallback to dateutil
                merged_df[col] = merged_df[col].apply(parse)
                parsed_date_columns.append(col)
            except:
                continue

print(f"Successfully parsed {len(parsed_date_columns)} columns as datetime:")
print(parsed_date_columns)

Successfully parsed 1 columns as datetime:
['EXAMDT']


#### Option 01: Convert them to (month + year) columns

In [167]:
# Find all datetime columns
datetime_cols = merged_df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns
print(f"Number of datetime columns: {len(datetime_cols)}\n")


# Create new columns in a separate DataFrame
new_cols = {}

for col in datetime_cols:
    new_cols[f'{col}_YEAR'] = merged_df[col].dt.year
    new_cols[f'{col}_MONTH'] = merged_df[col].dt.month

# Concatenate new columns all at once
merged_df = pd.concat([merged_df, pd.DataFrame(new_cols, index=merged_df.index)], axis=1)

# Drop the original datetime columns
merged_df.drop(columns=datetime_cols, inplace=True)

Number of datetime columns: 1



#### Option 02: Drop them

In [141]:
datetime_cols = merged_df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns
print(f"Number of datetime columns: {len(datetime_cols)}\n")

merged_df.drop(columns=parsed_date_columns, inplace=True)

Number of datetime columns: 1



### 5. Encode categorical columns

In [168]:
# Columns to encode
object_cols = merged_df.select_dtypes(include='object').columns.drop(['PATNO', 'EVENT_ID', 'NHY'], errors='ignore')

print(f"Number of object columns to encode: {len(object_cols)}")

# Fit and transform with OrdinalEncoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
merged_df[object_cols] = encoder.fit_transform(merged_df[object_cols].astype(str))

merged_df.head()

Number of object columns to encode: 1


,PATNO,EVENT_ID,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,...,SCAU23,SCAU23A,SCAU24,SCAU25,SCAU26A,SCAU26B,SCAU26C,SCAU26D,EXAMDT_YEAR,EXAMDT_MONTH
1,3001,V10,1,2.0,1.0,0.0,2.0,0.0,0.0,0.0,...,9.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0,2015,4
3,3001,V12,1,2.0,1.0,0.0,2.0,0.0,0.0,0.0,...,9.0,0.0,NaN,NaN,1.0,1.0,1.0,1.0,2016,6
5,3001,V14,1,2.0,0.0,0.0,2.0,0.0,1.0,0.0,...,9.0,0.0,NaN,NaN,1.0,1.0,1.0,1.0,2018,3
7,3001,V15,1,3.0,0.0,0.0,2.0,0.0,1.0,0.0,...,3.0,0.0,NaN,NaN,1.0,1.0,0.0,1.0,2019,3
9,3001,V17,1,2.0,1.0,1.0,3.0,1.0,1.0,0.0,...,9.0,0.0,9.0,9.0,1.0,1.0,0.0,1.0,2021,9


### 6. Normalize

In [169]:
# Identify all numeric columns
num_cols = merged_df.select_dtypes(include=['float64', 'int64', 'int32']).columns.tolist()
# Exclude PATNO and NHY from normalization
num_cols = [col for col in num_cols if col not in ['PATNO', 'EVENT_ID', 'NHY']]
print(f"Number of numerical columns to normalize: {len(num_cols)}")

# Fit and transform the data
merged_df[num_cols] = StandardScaler().fit_transform(merged_df[num_cols])

merged_df.head()


Number of numerical columns to normalize: 123


,PATNO,EVENT_ID,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,...,SCAU23,SCAU23A,SCAU24,SCAU25,SCAU26A,SCAU26B,SCAU26C,SCAU26D,EXAMDT_YEAR,EXAMDT_MONTH
1,3001,V10,-0.244827,0.705386,0.534785,-0.964417,0.074607,-0.387433,-0.930064,-0.374929,...,1.738097,-0.441942,NaN,NaN,-0.637679,2.107244,1.384309,1.330183,-1.058710,-0.694740
3,3001,V12,-0.244827,0.705386,0.534785,-0.964417,0.074607,-0.387433,-0.930064,-0.374929,...,1.738097,-0.441942,NaN,NaN,1.568187,2.107244,1.384309,1.330183,-0.790016,-0.098358
5,3001,V14,-0.244827,0.705386,-0.736217,-0.964417,0.074607,-0.387433,0.203986,-0.374929,...,1.738097,-0.441942,NaN,NaN,1.568187,2.107244,1.384309,1.330183,-0.252627,-0.992931
7,3001,V15,-0.244827,1.790230,-0.736217,-0.964417,0.074607,-0.387433,0.203986,-0.374929,...,0.074691,-0.441942,NaN,NaN,1.568187,2.107244,-0.722382,1.330183,0.016067,-0.992931
9,3001,V17,-0.244827,0.705386,0.534785,0.086328,1.080939,1.709294,0.203986,-0.374929,...,1.738097,-0.441942,0.863766,0.867444,1.568187,2.107244,-0.722382,1.330183,0.553455,0.796215


### 7. compute correlation

In [170]:
# Compute correlation matrix (absolute values)
corr_matrix = merged_df.corr(numeric_only=True).abs()

# Create a boolean mask for the upper triangle (to avoid double-checking pairs)
upper_tri = np.triu(corr_matrix, k=1)

# Find column pairs with correlation > 0.5
to_drop = [column for column in corr_matrix.columns if any(upper_tri[:, corr_matrix.columns.get_loc(column)] > 0.5)]

# Ensure we don't drop key columns
protected = {"PATNO", "EVENT_ID", "NHY"}
to_drop = [col for col in to_drop if col not in protected]
print(f"Original shape: {merged_df.shape}")

# Drop those columns from the dataframe
merged_df.drop(columns=to_drop, inplace=True)

print(f"Reduced shape: {merged_df.shape}")
print("Dropped columns due to high correlation:")
print(to_drop)


Original shape: (4114, 132)
Reduced shape: (4114, 87)
Dropped columns due to high correlation:
['ESS2', 'NP1ANXS', 'NP1RTOT', 'PDMEDYN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL', 'NP3TTAPL', 'NP3LGAGR', 'NP3LGAGL', 'NP3GAIT', 'NP3FRZGT', 'NP3PSTBL', 'NP3POSTR', 'NP3KTRML', 'NP3RTARU', 'NP3RTARL', 'NP3RTALL', 'NP3TOT', 'NP4DYSKI', 'NP4FLCTI', 'NP4FLCTX', 'NP4TOT', 'NP2DRES', 'NP2HYGN', 'NP2HOBB', 'NP2TURN', 'NP2PTOT', 'DRMFIGHT', 'DRMUMV', 'SCAU1', 'SCAU2', 'SCAU3', 'SCAU6', 'SCAU9', 'SCAU10', 'SCAU12', 'SCAU13', 'SCAU15', 'SCAU18', 'SCAU23', 'SCAU24', 'SCAU25', 'EXAMDT_YEAR']


### 8. Export to a csv file

In [171]:
# Move 'NHY' to the end
nhy_col = merged_df.pop('NHY')
merged_df['NHY'] = nhy_col

In [172]:
output_path = "Dataset/CSV/testing_for_memoire.csv"
merged_df.to_csv(output_path, index=False)

print(merged_df.shape)
print(merged_df.columns)

(4114, 87)
Index(['PATNO', 'EVENT_ID', 'ESS1', 'ESS3', 'ESS4', 'ESS5', 'ESS6', 'ESS7',
       'ESS8', 'NP1COG', 'NP1HALL', 'NP1DPRS', 'NP1APAT', 'NP1DDS', 'PDTRTMNT',
       'PDSTATE', 'HRPOSTMED', 'DBSYN', 'NP3SPCH', 'NP3FACXP', 'NP3RIGN',
       'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR', 'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL',
       'NP3TTAPR', 'NP3RISNG', 'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR',
       'NP3RTALU', 'NP3RTALJ', 'NP3RTCON', 'DYSKPRES', 'NP4WDYSK', 'NP4OFF',
       'NP4DYSTN', 'NP2SPCH', 'NP2SALV', 'NP2SWAL', 'NP2EAT', 'NP2HWRT',
       'NP2TRMR', 'NP2RISE', 'NP2WALK', 'NP2FREZ', 'DRMVIVID', 'DRMAGRAC',
       'DRMNOCTB', 'SLPLMBMV', 'SLPINJUR', 'DRMVERBL', 'DRMOBJFL', 'MVAWAKEN',
       'DRMREMEM', 'SLPDSTRB', 'STROKE', 'HETRA', 'PARKISM', 'RLS', 'NARCLPSY',
       'DEPRS', 'EPILEPSY', 'BRNINFM', 'CNSOTH', 'SCAU4', 'SCAU5', 'SCAU7',
       'SCAU8', 'SCAU11', 'SCAU14', 'SCAU16', 'SCAU17', 'SCAU19', 'SCAU20',
       'SCAU21', 'SCAU22', 'SCAU23A', 'SCAU26A', 'SCAU26B', 'SCAU